In [1]:
import os
import numpy as np

In [2]:
files = os.listdir()
d = {}
count = 0

In [3]:
X = None
Y = []

In [4]:
for f in files:
    if f.endswith(".npy"):
        data = np.load(f)
        d[count] = f[:-4]
        
        if X is None:
            X = data
            target = count*np.ones((data.shape[0]))
            Y.append(target)
            
        else:
            X = np.vstack((X, data))
            target = count*np.ones(data.shape[0])
            Y.append(target)
            
        count = count+1

In [5]:
Y = np.array(Y)
Y = Y.reshape((len(d)*20,))

In [6]:
def dist(a1, a2):
    return np.sum((a1-a2)**2)**.5

In [7]:
def knn(X, Y, q, k=5):
    m = X.shape[0]
    l = []
    
    for i in range(m):
        d = dist(q, X[i])
        l.append((d, Y[i]))
        
    l.sort()
    l = np.array(l[:k])
    l = l[:1]
    
    uniq, freq = np.unique(l, return_counts = True)
    p = np.argmax(freq)
    pred = uniq[p]
    return int(pred)

In [8]:
import cv2
import numpy as np

In [9]:
camera = cv2.VideoCapture(0)
face_detector = cv2.CascadeClassifier("FaceCascade/templatedata.xml")

In [ ]:
while True:
    ret, img = camera.read()
    if ret==False:
        continue
    
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    if len(faces)==0:
        continue
        
    face = faces[0]
    x, y, w, h = face
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 5)
    
    cropped_img = img[y:y+h, x:x+w, :]
    cropped_img = cv2.resize(cropped_img, (100, 100))
    
    label = knn(X, Y, cropped_img, 7)
    name = d[label]
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, name, (x, y-10), font, 1, (0, 255, 0), 4, cv2.LINE_AA)
    cv2.imshow("Face_Recognition", img)
    
    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
        
camera.release()
cv2.destroyAllWindows()